# Extracting Data for Country by ISO3

The Fathom flood data are stored as individual tiles, organized into folders per model on the GOST AWS bucket. We have generated virtual rasters (.vrt) for each model, making reading and extracting easier.

In [2]:
import sys
import os
import rasterio

import geopandas as gpd

sys.path.insert(0, "../../src")

import GOSTrocks.dataMisc as dMisc
import GOSTrocks.rasterMisc as rMisc
from GOSTrocks.misc import tPrint

In [5]:
os.path.abspath("../../src")

'c:\\WBG\\Work\\Code\\GOSTrocks\\src'

In [2]:
iso3 = "KEN"
out_folder = f"/home/wb411133/temp/FATHOM/{iso3}"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

# This demo uses the default national boundaries included with GeoPandas, but this can be changed here
world_filepath = gpd.datasets.get_path("naturalearth_lowres")
world = gpd.read_file(world_filepath)
inB = world.loc[world["iso_a3"] == iso3].copy()

In [4]:
# Select layer to download
flood_type = ["COASTAL", "FLUVIAL", "PLUVIAL"]
defence = ["DEFENDED"]
return_period = ["1in5", "1in10", "1in50"]
climate_model = ["PERCENTILE50"]
year = ["2020"]

# all_vrts is a pandas dataframe with all the vrt paths to the global datasets, with columns defining
# the various models' defining attributes
all_vrts = dMisc.get_fathom_vrts(True)
sel_images = all_vrts.loc[
    (all_vrts["FLOOD_TYPE"].isin(flood_type))
    & (all_vrts["DEFENCE"].isin(defence))
    & (all_vrts["RETURN"].isin(return_period))
    & (all_vrts["CLIMATE_MODEL"].isin(climate_model))
]

In [4]:
all_vrts.head()

,RETURN,FLOOD_TYPE,DEFENCE,DEPTH,YEAR,CLIMATE_MODEL,PATH
0,1in10,COASTAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
1,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP1_2.6,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
2,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP2_4.5,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
3,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP3_7.0,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
4,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP5_8.5,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...


In [5]:
sel_images.head()

,RETURN,FLOOD_TYPE,DEFENCE,DEPTH,YEAR,CLIMATE_MODEL,PATH
0,1in10,COASTAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
26,1in10,FLUVIAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
52,1in10,PLUVIAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
325,1in5,COASTAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
351,1in5,FLUVIAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...


In [6]:
# For each image in the selected images dataframe, we clip out the area of interest
#     which is defined by the ioso3 code, but could be any GeoDataFrame

for idx, row in sel_images.iterrows():
    out_file = os.path.join(out_folder, os.path.basename(row["PATH"]))
    if not os.path.exists(out_file):
        cur_r = rasterio.open(row["PATH"])
        rMisc.clipRaster(cur_r, inB, out_file)
    tPrint(os.path.basename(row["PATH"]))

08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in10-FLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in10-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in5-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in5-FLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in5-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in50-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in50-FLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
08:31:04	GLOBAL-1ARCSEC-NW_OFFSET-1in50-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
